In [ ]:
import os
import mysql.connector
from mysql.connector import Error

DB_CONFIG = {
    'host': os.getenv('DB_HOST', 'localhost'),
    'port': int(os.getenv('DB_PORT', 3306)),
    'user': os.getenv('DB_USER', 'root'),
    'password': os.getenv('DB_PASSWORD', ''),
    'database': os.getenv('DB_NAME', 'exametrics'),  # Changed 'db' to 'database' for mysql.connector
    'pool_size': int(os.getenv('DB_POOL_SIZE', 5))  # Changed 'maxsize' to 'pool_size'
}

def test_database_connection():
    try:
        # Establish a connection
        connection = mysql.connector.connect(**DB_CONFIG)
        
        if connection.is_connected():
            print("Successfully connected to the database")
            
            # Check if results table exists
            cursor = connection.cursor()
            cursor.execute("""
                SELECT COUNT(*) 
                FROM information_schema.tables 
                WHERE table_schema = %s 
                AND table_name = 'results'
            """, (DB_CONFIG['database'],))
            
            if cursor.fetchone()[0] == 1:
                print("Results table exists")
                # Optional: Count rows in results table
                cursor.execute("SELECT COUNT(*) FROM results")
                count = cursor.fetchone()[0]
                print(f"Results table has {count} rows")
            else:
                print("Results table does not exist")
                
    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if 'connection' in locals() and connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

if __name__ == "__main__":
    test_database_connection()